In [1]:
import pandas as pd

korean_air_data = pd.read_csv("korean_air_data/korean_air_2020_01.csv")

korean_air_data 

korean_air_data.columns = ["a","title","text","editor","times","date"]

del korean_air_data["a"]

# na값 제거
data = korean_air_data.dropna()

# 불용어 지정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','박정환','도','을','것','를','등','으로','자','에','등','와','한','하다''것','\n','로']

# 한글 외의 모든 데이터 제거 : 기업명이 영어로 되어있는 경우를 어떻게 처리 해야할까?
data = data.text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

okt = Okt()
X_train = []

for sen in data :
    temp_X = []
    
    # 토큰화
    temp_X = okt.morphs(sen, stem = True) 
    
    # 불용어 제거
    temp_X = [word for word in temp_X if not word in stopwords] 
    
    X_train.append(temp_X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
vocab_size = total_cnt - rare_cnt
print('단어 집합의 크기 :',vocab_size)
    

단어 집합(vocabulary)의 크기 : 8648
등장 빈도가 2번 이하인 희귀 단어의 수: 4170
단어 집합에서 희귀 단어의 비율: 48.21924144310824
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.497673817523908
단어 집합의 크기 : 4478


In [14]:
nouns = []
str = ''

for k in range(0,len(X_train)):
    for i in range(0,len(X_train[k])):
        str = str + X_train[k][i] + ' '
    nouns.append(str)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_df = pd.DataFrame()

tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfidf = tfIdfVectorizer.fit_transform(nouns)

for i in range(0,len(X_train)):
    df = pd.DataFrame(tfidf[i].T.toarray(), index=tfIdfVectorizer.get_feature_names(), columns=["text{}".format(i)])
    tfidf_df = pd.concat([tfidf_df,df],axis = 1)

In [16]:
X_train_tfidf = []

for i in range(0,len(X_train)):
    idx = tfidf_df['text{}'.format(i)] > 0.01
    result = tfidf_df[idx].index.tolist()
    X_train_tfidf.append(result)

len(X_train_tfidf)

432

In [2]:
from konlpy.corpus import kolaw
from konlpy.tag import Okt
from nltk.collocations import BigramAssocMeasures
from nltk.collocations import TrigramAssocMeasures
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import TrigramCollocationFinder
from collections import Counter

ngram = [(BigramAssocMeasures(),BigramCollocationFinder),
         (TrigramAssocMeasures(),TrigramCollocationFinder)]

okt= Okt()# loading tagger
nouns = []
for sen in data:
    token = okt.morphs(sen, stem = True) # get nouns
    token = [word for word in token if not word in stopwords] 
    nouns.append(token)
    


In [3]:
founds_from_4measure = []
X_train_ngram = []

for i in range(0,len(nouns)):
    for measure, finder in ngram:
        finder = finder.from_words(nouns[i])
        founds = finder.nbest(measure.pmi, 10)       # pmi - 상위 30개 추출
        founds += finder.nbest(measure.chi_sq, 10)   # chi_sq - 상위 30개 추출
        founds += finder.nbest(measure.mi_like, 10)  # mi_like - 상위 30개 추출
        founds += finder.nbest(measure.jaccard, 10)  # jaccard - 상위 30개 추출

        founds_from_4measure += founds

        collocations = [' '.join(collocation) for collocation in founds_from_4measure]
        collocations = [(w) for w,f in Counter(collocations).most_common() if f > 2]
        
    X_train_ngram.append(collocations)


In [4]:
X_train_pre = []

for i in range(0,len(X_train)):
    X_train_pre.append(X_train_g[i] + X_train_ngram[i])
